In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')

In [3]:
## droping unimportant columns
data = data.drop(['RowNumber', 'CustomerId','Surname'], axis=1)

In [4]:
# Lets encode the gender first
label_encoder_gender = LabelEncoder()
data['Gender']= label_encoder_gender.fit_transform(data['Gender'])
data['Gender']

0       0
1       0
2       0
3       0
4       0
       ..
9995    1
9996    1
9997    0
9998    1
9999    0
Name: Gender, Length: 10000, dtype: int32

In [6]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoded=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
# Splitting data
X= data.drop('EstimatedSalary', axis=1)
y=data['EstimatedSalary']

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
##Scale down this feature with standard scalar
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [14]:
## Save the encoders and scaler
with open('reg_label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('reg_onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open('reg_scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

Training ANN Regression

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [17]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1) # Default activation is linear activation (Regression)
])

In [21]:
model.compile(optimizer='adam', loss='mean_absolute_error', metrics='mae')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 


Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
log_dir="reg_logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callsback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [23]:
## Set early stopping - if model stops learning apply early stopping. If loss is not decreasing after certain epoch ---stop further training
early_stopping_callback=EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

In [24]:
## Train the model
history=model.fit(
    X_train, y_train,validation_data=(X_test,y_test), epochs=100,
    callbacks=[tensorflow_callsback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 13s 21ms/step - loss: 100363.8203 - mae: 100363.8203 - val_loss: 98467.2812 - val_mae: 98467.2812
Epoch 2/100
250/250 [==============================] - 3s 14ms/step - loss: 99464.0000 - mae: 99464.0000 - val_loss: 96662.4688 - val_mae: 96662.4688
Epoch 3/100
250/250 [==============================] - 3s 14ms/step - loss: 96341.2500 - mae: 96341.2500 - val_loss: 92093.5156 - val_mae: 92093.5156
Epoch 4/100
250/250 [==============================] - 3s 13ms/step - loss: 90277.7734 - mae: 90277.7734 - val_loss: 84652.1719 - val_mae: 84652.1719
Epoch 5/100
250/250 [==============================] - 4s 15ms/step - loss: 81629.9219 - mae: 81629.9219 - val_loss: 75258.8906 - val_mae: 75258.8906
Epoch 6/100
250/250 [==============================] - 4s 14ms/step - loss: 71864.9609 - mae: 71864.9609 - val_loss: 66186.4219 - val_mae: 66186.4219
Epoch 7/100
250/250 [==============================] - 4s 14ms/step - loss: 62935.8828 - mae: 6

In [25]:
%load_ext tensorboard

In [27]:
%tensorboard --logdir reg_logs/fit

Reusing TensorBoard on port 6006 (pid 29632), started 0:00:27 ago. (Use '!kill 29632' to kill it.)

In [29]:
# Evaluate model on Test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'TEST MAE :{test_mae}')

63/63 [==============================] - 1s 6ms/step - loss: 50302.9258 - mae: 50302.9258
TEST MAE :50302.92578125


In [30]:
model.save('reg_model.h5')

m:\Learning_Workspace\AI\AI_Projects\KrishNaikProjects\ANN-DL-Algorithm\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
